<a href="https://colab.research.google.com/github/fernandoariel/lodefernando/blob/master/TP_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train.csv')
df.sample(10)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
319063,32,135378,Friday,4.900003e+09,1,DSD GROCERY,9205.0
405453,40,172072,Friday,5.400015e+09,1,HOUSEHOLD PAPER GOODS,3004.0
146401,42,61491,Sunday,7.232616e+10,1,SHOES,2538.0
440478,39,186845,Sunday,7.910093e+09,1,PETS AND SUPPLIES,806.0
188718,37,79882,Wednesday,3.700087e+09,1,HOUSEHOLD PAPER GOODS,3050.0
108866,40,47624,Friday,7.225004e+09,1,COMM BREAD,3702.0
144957,40,60971,Sunday,1.657191e+09,1,DSD GROCERY,3858.0
347098,33,146720,Sunday,3.500053e+09,1,HOUSEHOLD CHEMICALS/SUPP,55.0
49236,32,19512,Sunday,3.700028e+09,1,INFANT CONSUMABLE HARDLINES,9010.0
317411,40,134666,Friday,3.940002e+09,2,GROCERY DRY GOODS,2828.0


In [37]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType

    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # the next three operations are the ones we have just presented in the previous lines
    
    # drop the columns we won't use (it may be good to use them somehow)
    df = df.drop(["Upc", "FinelineNumber"], axis=1)

    # one-hot encoding for the DepartmentDescription
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)

    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)

       

    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

In [39]:
X, y, XX, yy = transform_data("/content/gdrive/My Drive/Colab Notebooks/train.csv", "/content/gdrive/My Drive/Colab Notebooks/test.csv")

In [40]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it could be useful for you depending on your approach)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [41]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

# **Random Forest con RandomizedSearchCV**

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

In [49]:
param_distributions = {'criterion': ['gini', 'entropy'], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 6], 'n_estimators': [30,60], 'max_features':['sqrt','log2',6]}
forest_clf=RandomizedSearchCV( RandomForestClassifier(random_state=1) ,param_distributions, n_iter=10, scoring='accuracy', cv=3)


forest_clf.fit(X_train, y_train)
best_forest_clf=forest_clf.best_estimator_

In [50]:
print('random forest accuracy: ', forest_clf.best_score_)
print(best_forest_clf)
results = results.append({'clf': best_forest_clf, 'best_acc': forest_clf.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

random forest accuracy:  0.6635763000852516
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=60,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
The best classifier so far is: 
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fract

In [51]:
X.shape, XX.shape

((67029, 79), (28645, 79))

In [56]:
yy = results.clf.iloc[0].predict(XX)
yy.shape

(28645,)

In [58]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])
submission

,VisitNumber,TripType
0,7,9
1,8,40
2,15,21
3,19,25
4,23,8
...,...,...
28640,191331,9
28641,191335,32
28642,191342,8
28643,191345,39


In [ ]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)

# **Random Forest sin grilla de parametros**

In [42]:
model=RandomForestClassifier(n_estimators=500, random_state=2, min_samples_leaf=100, min_samples_split=10, oob_score=True, n_jobs=-1)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=100, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=True, random_state=2, verbose=0,
                       warm_start=False)

In [43]:
y_predicted=model.predict(XX)
model.score(X_valid, y_valid)

0.6055994828186384

In [44]:
y_predicted

array([ 8, 39, 21, ...,  9, 39, 39])

In [45]:
X.shape, XX.shape

((67029, 79), (28645, 79))

In [ ]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, y_predicted)), columns=["VisitNumber", "TripType"])

In [ ]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)

# **SGDClassifier**

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterSampler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
grid={'loss': [ 'hinge', 'log', 'modified_huber'] ,       
    'alpha': [0.001, 0.01, 0.1],
    'learning_rate': [  'constant', 'optimal'  ],
    'eta0': [1e-1, 1  ]
    
}

In [ ]:
model_2 = SGDClassifier(random_state = 2)

clf_2 = RandomizedSearchCV(model_2, param_distributions=grid, scoring='accuracy', cv=2)

clf_2.fit(X_train, y_train);

In [ ]:
best_clf=clf_2.best_estimator_

In [ ]:
print('random forest accuracy: ', clf_2.best_score_)
print(best_clf)
results = results.append({'clf': best_clf, 'best_acc': clf_2.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

random forest accuracy:  0.05253623188405797
SGDClassifier(alpha=0.1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=1, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='modified_huber', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=2, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)
The best classifier so far is: 
SGDClassifier(alpha=0.1, average=False, class_weight=None, early_stopping=False,
              epsilon=0.1, eta0=1, fit_intercept=True, l1_ratio=0.15,
              learning_rate='optimal', loss='modified_huber', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=2, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)


In [ ]:
#Nota: los resultados son muy pobres, no vale la pena seguir probando

In [ ]:
yy = results.clf.iloc[0].predict(XX)

In [ ]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [ ]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)

0    SGDClassifier(alpha=0.1, average=False, class_...
Name: clf, dtype: object

# **Boosting**

In [63]:
from xgboost import XGBClassifier

In [86]:
model_3=XGBClassifier(n_estimators=600,random_state=2)
model_3.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [87]:
y_pred=model_3.predict(XX)

In [79]:
#XGB con valores por defecto mejoró el score
#al aumentar  n_estimators=700 y random_state=2 mejoró aun más, tambien el tiempo de ejecución (-.-)!
#aumentar n_estimators=800 redujo el score

In [88]:
# evaluate predictions
model_3.score(X_valid, y_valid)

0.6958078472325824

In [89]:
X.shape, XX.shape


((67029, 79), (28645, 79))

In [90]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, y_pred)), columns=["VisitNumber", "TripType"])

In [91]:
submission.to_csv("/content/gdrive/My Drive/Colab Notebooks/submission.csv", header=True, index=False)